In [1]:
%matplotlib inline

In [47]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, GRU
from keras.layers import Activation
from keras.layers import Masking
from keras.layers import Lambda
from keras.optimizers import RMSprop
from keras import backend as k
from sklearn.preprocessing import normalize

In [7]:
import pandas as pd

In [51]:
def weibull_loglike_discrete(y_true, ab_pred, name=None):
    y_ = y_true[:, 0]
    u_ = y_true[:, 1]
    a_ = ab_pred[:, 0]
    b_ = ab_pred[:, 1]
    
    hazard0 = k.pow((y_ + 1e-35) / a_, b_)
    hazard1 = k.pow((y_ + 1) / a_, b_)
    
    return -1 * k.mean(u_ * k.log(k.exp(hazard1 - hazard0) - 1.0) - hazard1)

In [43]:
def activate(ab):
    a = k.exp(ab[:, 0])
    b = k.softplus(ab[:, 1])
    
    a = k.reshape(a, (k.shape(a)[0], 1))
    b = k.reshape(b, (k.shape(b)[0], 1))
    
    return k.concatenate((a, b), axis=1)

In [5]:
def load_file(name):
    with open(name, 'r') as file:
        return np.loadtext(file, delimiter=',')

In [9]:
id_col = 'unit_number'
time_col = 'time'
feature_cols = [ 'op_setting_1', 'op_setting_2', 'op_setting_3'] + ['sensor_measurement_{}'.format(x) for x in range(1,22)]
column_names = [id_col, time_col] + feature_cols

In [10]:
np.set_printoptions(suppress=True, threshold=10000)

train_orig = pd.read_csv('https://raw.githubusercontent.com/daynebatten/keras-wtte-rnn/master/train.csv', header=None, names=column_names)
test_x_orig = pd.read_csv('https://raw.githubusercontent.com/daynebatten/keras-wtte-rnn/master/test_x.csv', header=None, names=column_names)
test_y_orig = pd.read_csv('https://raw.githubusercontent.com/daynebatten/keras-wtte-rnn/master/test_y.csv', header=None, names=['T'])

In [11]:
from sklearn import pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [14]:
all_data_original = pd.concat([train_orig, test_x_orig])

scaler = pipeline.Pipeline(steps=[('minmax', MinMaxScaler(feature_range=(-1, 1))),
                                 ('remove_constant', VarianceThreshold())
                                 ])
all_data = all_data_original.copy()
all_data = np.concatenate([all_data[['unit_number', 'time']], scaler.fit_transform(all_data[feature_cols
                                                                                           ])], axis=1)

In [15]:
train = all_data[0:train_orig.shape[0], :]
test = all_data[train_orig.shape[0]:, :]

train[:, 0:2] -= 1
test[:, 0:2] -= 1

In [21]:
def build_data(engine, time, x, max_time, is_test, mask_value):
    out_y = []
    
    d = x.shape[1]
    
    out_x = []
    
    n_engines = 100
    for i in range(n_engines):
        max_engine_time = int(np.max(time[engine == i])) + 1
        
        if is_test:
            start = max_engine_time -1 
        else:
            start = 0
        
        this_x = []
        
        for j in range(start, max_engine_time):
            engine_x = x[engine == i]
            out_y.append(np.array((max_engine_time-j, 1), ndmin=2))
            
            xtemp = np.zeros((1, max_time, d))
            xtemp += mask_value
            
            xtemp[:, max_time-min(j,99)-1:max_time,:] = engine_x[max(0,j-max_time+1):j+1,:]
            this_x.append(xtemp)
        this_x = np.concatenate(this_x)
        out_x.append(this_x)
    out_x = np.concatenate(out_x)
    out_y = np.concatenate(out_y)
    return out_x, out_y

In [22]:
max_time = 100
mask_value = -99

train_x, train_y = build_data(engine=train[:,0], time=train[:,1], x=train[:,2:], 
                              max_time=max_time, is_test=False, mask_value=mask_value)
test_x, _ = build_data(engine=test[:,0], time=test[:,1], x=test[:,2:],
                      max_time=max_time, is_test=True, mask_value=mask_value)

In [55]:
test_y = test_y_orig.copy()
test_y['E'] = 1

In [56]:
print('train_x', train_x.shape,'train_y',train_y.shape, 'test_x', test_x.shape, 'test_y', test_y.shape)

train_x (20631, 100, 17) train_y (20631, 2) test_x (100, 100, 17) test_y (100, 2)


In [33]:
tte_mean_train = np.nanmean(train_y[:,0])
mean_u = np.nanmean(train_y[:,1])

init_alpha = -1.0/np.log(1.0 - 1.0/(tte_mean_train+1.0))
init_alpha = init_alpha/mean_u

print('tte_mean_train', tte_mean_train, 'init_alpha: ',init_alpha,'mean uncensored train: ',mean_u)

tte_mean_train 108.80786195530997 init_alpha:  109.30709957832491 mean uncensored train:  1.0


In [52]:
n_features = train_x.shape[-1]

model = Sequential()
model.add(Masking(mask_value=mask_value, input_shape=(None, n_features)))
model.add(GRU(20, activation='tanh', recurrent_dropout=0.25))

# Prediction layer for alph and beta of Weibull distribution
model.add(Dense(2))

# Apply custom activation function for alpha and beta
model.add(Activation(activate))
model.compile(loss=weibull_loglike_discrete, optimizer=RMSprop(lr=0.001))

In [53]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_9 (Masking)          (None, None, 17)          0         
_________________________________________________________________
gru_9 (GRU)                  (None, 20)                2280      
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 42        
_________________________________________________________________
activation_7 (Activation)    (None, 2)                 0         
Total params: 2,322
Trainable params: 2,322
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.fit(train_x, train_y,
         epochs=1,
         batch_size=100,
         verbose=1,
         validation_data=(test_x, test_y)
         )

Train on 20631 samples, validate on 100 samples
Epoch 1/1
20631/20631 [==============================] - 42s 2ms/step - loss: 6.8617 - val_loss: 4.6776


In [58]:
model.fit(train_x, train_y,
         epochs=100,
         batch_size=100,
         verbose=1,
         validation_data=(test_x, test_y)
         )

Train on 20631 samples, validate on 100 samples
Epoch 1/100
20631/20631 [==============================] - 33s 2ms/step - loss: 4.9681 - val_loss: 4.5260
Epoch 2/100
20631/20631 [==============================] - 29s 1ms/step - loss: 4.8619 - val_loss: 4.4657
Epoch 3/100
20631/20631 [==============================] - 28s 1ms/step - loss: 4.7929 - val_loss: 4.2924
Epoch 4/100
20631/20631 [==============================] - 28s 1ms/step - loss: 4.7201 - val_loss: 4.3618
Epoch 5/100
20631/20631 [==============================] - 29s 1ms/step - loss: 4.6722 - val_loss: 4.2719
Epoch 6/100
20631/20631 [==============================] - 28s 1ms/step - loss: 4.6344 - val_loss: 4.2467
Epoch 7/100
20631/20631 [==============================] - 28s 1ms/step - loss: 4.5954 - val_loss: 4.1229
Epoch 8/100
20631/20631 [==============================] - 30s 1ms/step - loss: 4.5706 - val_loss: 4.2826
Epoch 9/100
20631/20631 [==============================] - 30s 1ms/step - loss: 4.5472 - val_loss: 4.059

20631/20631 [==============================] - 29s 1ms/step - loss: 4.1314 - val_loss: 4.1300
Epoch 78/100
20631/20631 [==============================] - 30s 1ms/step - loss: 4.1305 - val_loss: 4.1708
Epoch 79/100
20631/20631 [==============================] - 30s 1ms/step - loss: 4.1265 - val_loss: 4.1302
Epoch 80/100
20631/20631 [==============================] - 30s 1ms/step - loss: 4.1209 - val_loss: 4.3246
Epoch 81/100
20631/20631 [==============================] - 29s 1ms/step - loss: 4.1192 - val_loss: 4.1266
Epoch 82/100
20631/20631 [==============================] - 30s 1ms/step - loss: 4.1126 - val_loss: 4.4299
Epoch 83/100
20631/20631 [==============================] - 29s 1ms/step - loss: 4.1118 - val_loss: 4.2292
Epoch 84/100
20631/20631 [==============================] - 29s 1ms/step - loss: 4.1060 - val_loss: 4.4123
Epoch 85/100
20631/20631 [==============================] - 30s 1ms/step - loss: 4.1072 - val_loss: 4.2373
Epoch 86/100
20631/20631 [========================

In [59]:
test_predict = model.predict(test_x)
test_predict = np.resize(test_predict, (100,2))
test_result = np.concatenate((test_y, test_predict), axis=1)

In [ ]:
test_result_df = pd.DataFramerame(test_result, columns=['T','E', 'alpha','beta'])
test_result_df['unit_number'] = np.arange(1, test_results_df.shape[0]+1)

In [ ]:
test_result_df